# Create simple chloropleth maps by ecoregion, state, and watershed

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib


## Function for making maps

In [ ]:
def make_map(df, shape, shape_aea, merge_column, title, density=True, log=True, axs=[]):
    df = df.assign(nonzero=df['area']>0)
    df = df.groupby('reg').sum()
    df['area'] = df['area']*0.01 # Convert to ha

    shape[merge_column] = shape[merge_column].astype(int)
    shape_aea[merge_column] = shape_aea[merge_column].astype(int)
    shape_aea['reg_area_m2'] = shape_aea.area
    
    shape = shape.merge(shape_aea[[merge_column, 'reg_area_m2']], on=merge_column, )
    shape = shape.merge(df, left_on=merge_column, right_on='reg')

    if density:
        # Calculate density in ha/ha or count/ha
        # In denom: divide by pixel size^2 to conver to 100m^2, then *0.01 to convert to ha
        shape['area'] = shape['area']/(0.0001*shape['reg_area_m2'])
        shape['nonzero'] = shape['nonzero']/(0.0001*shape['reg_area_m2']) 
#         # In denom: divide by pixel size^2 to conver to 100m^2, then *0.01 to convert to ha
#         shape['area'] = shape['area']/(0.01*shape.area/(0.000089831528412**2))
#         shape['nonzero'] = shape['nonzero']/(0.01*shape.area/(0.000089831528412**2)) 
    # Make plots
    if len(axs) == 0:
        fig, axs = plt.subplots(1, 2, figsize=[12,6])
    if log:
        shape.plot(column='area', ax=axs[0], legend=True,
                   norm=matplotlib.colors.LogNorm(vmin=shape.loc[shape['area']>0,'area'].min(), vmax=shape['area'].max()))
    else:
        shape.plot(column='area', ax=axs[0], legend=True)
    shape.boundary.plot(ax=axs[0], linewidth=0.25)
    axs[0].set_title(title+' Reservoir Area Density\n(Reservoir Area Fraction)', size=15)
    axs[0].set_ylabel('Latitude', size=14)
    axs[0].set_xlabel('Longitude', size=14)
    axs[0].tick_params(axis='both', labelsize=12 )
    if log:    
        shape.plot(column='nonzero', ax=axs[1], legend=True,
                  norm=matplotlib.colors.LogNorm(vmin=shape.loc[shape['nonzero']>0,'nonzero'].min(), vmax=shape['nonzero'].max()))
    else:
        shape.plot(column='nonzero', ax=axs[1], legend=True)
    shape.boundary.plot(ax=axs[1], linewidth=0.25)
    axs[1].set_title(title+' Reservoir Count Density\n(Reservoirs per ha)', size=15)
    axs[1].set_ylabel('Latitude', size=14)
    axs[1].set_xlabel('Longitude', size=14)
    axs[1].tick_params(axis='both', labelsize=12 )


##  States

In [ ]:
state_df = pd.read_csv('./data/state_sizes_aea.csv')
# Remove two states that are barely in the study region
state_df = state_df.loc[~state_df['reg'].isin([11,15])]
# Regular shapefile is for plotting map
state_shape = gpd.read_file('../accuracy/data/shapefiles/brazil_states_clip.shp')
# AEA shapefile is for area calculation
state_shape_aea = gpd.read_file('../accuracy/data/shapefiles/brazil_states_aea.shp')
make_map(state_df, state_shape, state_shape_aea, 'GEOCODIGO', 'State', log=False)

# Biomes


In [ ]:
biome_df = pd.read_csv('./data/biomes_sizes_aea.csv')
# Drop 9 and 14, very small/barely in study region
biome_df = biome_df.loc[~biome_df['reg'].isin([13,14])]
# Regular shapefile is for plotting map
biome_shape = gpd.read_file('../accuracy/data/shapefiles/biome_clip.shp')
# AEA shapefile is for area calculation
biome_shape_aea = gpd.read_file('../accuracy/data/shapefiles/biome_clip_aea.shp')
make_map(biome_df, biome_shape, biome_shape_aea, 'BIOME', 'Biome', log=False)

## Watersheds


In [ ]:
water_df = pd.read_csv('./data/watersheds_sizes_aea.csv')
# Drop smallest watershed
# water_df = water_df.loc[~water_df['reg'].isin([7238])]
# Regular shapefile is for plotting map
water_shape = gpd.read_file('../accuracy/data/shapefiles/watersheds_4digit_clip.shp')
# AEA shapefile is for area calculation
water_shape_aea = gpd.read_file('../accuracy/data/shapefiles/watersheds_4digit_aea.shp')
make_map(water_df, water_shape, water_shape_aea, 'NUNIVOTTO4', 'Watershed', log=False)

# Full Figure

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=[12,15])
make_map(state_df, state_shape, state_shape_aea, 'GEOCODIGO', 'State', log=False, axs=axs[0])
make_map(biome_df, biome_shape, biome_shape_aea, 'BIOME', 'Biome', log=False, axs=axs[1])
make_map(water_df, water_shape, water_shape_aea, 'NUNIVOTTO4', 'Watershed', log=False, axs=axs[2])
fig.tight_layout() 
